<a href="https://colab.research.google.com/github/Ajiteshreddy7/Object-Detection/blob/main/crime_rate_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0.0 Spark Environment Setup

In [1]:
#pyspark setup

!apt-get update
# Install Java 8 (required by Spark)
!apt-get install openjdk-11-jdk-headless -qq > /dev/null

# Install Spark
!pip install pyspark

# setup environment variables
import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/usr/local/lib/python3.10/dist-packages/pyspark"

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,108 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,472 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/

# 0.1 Crime Data Generation

In [2]:
import pyspark
from pyspark.sql import SparkSession

# Create or get an existing Spark session
spark = SparkSession.builder.master("local[*]").appName("Crime-rate-prediction").getOrCreate()

In [3]:
import pandas as pd
import numpy as np

def generate_city_crime_data(num_cities=100, seed=42):
    """
    Generates synthetic data for city crime rates with logical correlations and occasional null values.

    Args:
    - num_cities (int): Number of cities to generate data for.
    - seed (int): Random seed for reproducibility.

    Returns:
    - pd.DataFrame: A DataFrame containing the generated data.
    """
    np.random.seed(seed)

    # Generate city names
    cities = [f"City{i}" for i in range(1, num_cities + 1)]

    # Generate population density (people per square mile)
    population_density = np.random.randint(500, 20000, size=num_cities)

    # Poverty rate (%) - Higher in densely populated cities
    poverty_rate = np.clip(15 + (population_density / 2000) + np.random.normal(0, 5, num_cities), 5, 30)

    # Police budget ($) - Correlated with population density
    police_budget = (population_density * 50) + np.random.randint(500000, 2000000, size=num_cities)

    # Unemployment rate (%) - Higher in cities with high poverty
    unemployment_rate = np.clip(5 + (poverty_rate / 3) + np.random.normal(0, 2, num_cities), 3, 15)

    # Housing quality index (1 to 10) - Inversely related to poverty rate
    housing_quality_index = np.clip(10 - (poverty_rate / 5) + np.random.normal(0, 1, num_cities), 1, 10)

    # Traffic violation rate (per 1000 people) - Higher in densely populated areas
    traffic_violation_rate = np.clip(100 + (population_density / 100) + np.random.normal(0, 30, num_cities), 50, 500)

    # Drug abuse rate (%) - Higher in cities with low housing quality
    drug_abuse_rate = np.clip(10 + (20 - housing_quality_index) + np.random.normal(0, 5, num_cities), 5, 40)

    # Mental health services availability (1 to 10) - Better in cities with higher police budgets
    mental_health_services = np.clip(5 + (police_budget / 200000) + np.random.normal(0, 2, num_cities), 1, 10)

    # Gun ownership rate (%) - Randomly distributed but generally higher in cities with lower population density
    gun_ownership_rate = np.clip(20 + (15000 / (population_density + 1)) + np.random.normal(0, 10, num_cities), 20, 60)

    # Crime rate (per 1000 people) - Correlated with poverty, unemployment, and drug abuse
    crime_rate = np.clip(20 + (poverty_rate * 0.8) + (unemployment_rate * 1.2) +
                         (drug_abuse_rate * 0.5) - (mental_health_services * 2) +
                         np.random.normal(0, 10, num_cities), 10, 100)

    # Introduce occasional null values randomly
    for col in ['poverty_rate', 'housing_quality_index', 'traffic_violation_rate']:
        null_indices = np.random.choice(num_cities, size=int(0.1 * num_cities), replace=False)
        locals()[col][null_indices] = np.nan

    # Create DataFrame
    data = pd.DataFrame({
        'city_name': cities,
        'population_density': population_density,
        'poverty_rate': poverty_rate,
        'police_budget': police_budget,
        'unemployment_rate': unemployment_rate,
        'housing_quality_index': housing_quality_index,
        'traffic_violation_rate': traffic_violation_rate,
        'drug_abuse_rate': drug_abuse_rate,
        'mental_health_services': mental_health_services,
        'gun_ownership_rate': gun_ownership_rate,
        'crime_rate': crime_rate
    })

    return data

# Generate and display the synthetic data
crime_data = generate_city_crime_data()

# Task 1: Data Preprocessing & Feature Engineering

Preview top 10 rows of data

In [4]:
crime_data.head(5)

,city_name,population_density,poverty_rate,police_budget,unemployment_rate,housing_quality_index,traffic_violation_rate,drug_abuse_rate,mental_health_services,gun_ownership_rate,crime_rate
0,City1,16295,27.321477,1317619,10.986045,NaN,301.213897,21.255302,10.000000,30.417632,66.092051
1,City2,1360,17.162807,1349474,14.721632,8.710644,110.853018,14.995146,8.745041,27.411433,46.255446
2,City3,5890,12.755851,1248851,6.851687,7.106241,187.012176,28.837764,10.000000,20.000000,34.298549
3,City4,12464,20.852981,2484028,14.353755,6.078718,207.560216,23.714740,10.000000,23.778102,62.900000
4,City5,11784,25.756818,1725784,13.885273,5.173155,230.251384,25.696677,10.000000,24.117530,60.217491


Find out missing or null values in columns

In [ ]:

missing_columns = crime_data.columns[crime_data.isnull().any()].tolist()

print(missing_columns)

['poverty_rate', 'housing_quality_index', 'traffic_violation_rate']


**Handle Missing Data**

In [10]:
# Impute missing values for numerical features with the median
crime_data['housing_quality_index'] = crime_data['housing_quality_index'].fillna(crime_data['housing_quality_index'].median())
crime_data['traffic_violation_rate'] = crime_data['traffic_violation_rate'].fillna(crime_data['traffic_violation_rate'].median())
crime_data['poverty_rate'] = crime_data['poverty_rate'].fillna(crime_data['poverty_rate'].median())

# Check if there are any remaining missing values
print("\nAfter imputing missing values:")
print(crime_data.info())



After imputing missing values:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 16 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   city_name                      100 non-null    object 
 1   population_density             100 non-null    int64  
 2   poverty_rate                   100 non-null    float64
 3   police_budget                  100 non-null    int64  
 4   unemployment_rate              100 non-null    float64
 5   housing_quality_index          100 non-null    float64
 6   traffic_violation_rate         100 non-null    float64
 7   drug_abuse_rate                100 non-null    float64
 8   mental_health_services         100 non-null    float64
 9   gun_ownership_rate             100 non-null    float64
 10  crime_rate                     100 non-null    float64
 11  estimated_income               100 non-null    float64
 12  income_to_police_bu

**Convert Percentage Strings to Numeric Values**

In [11]:
percent_columns = ['poverty_rate', 'unemployment_rate', 'drug_abuse_rate', 'gun_ownership_rate']

for col in percent_columns:
    # Remove the '%' symbol and convert to float
    crime_data[col] = crime_data[col].astype(str).str.replace('%', '').astype(float)

# Display the converted columns to confirm
print("\nAfter converting percentages to numeric values:")
print(crime_data[percent_columns].head())



After converting percentages to numeric values:
   poverty_rate  unemployment_rate  drug_abuse_rate  gun_ownership_rate
0     27.321477          10.986045        21.255302           30.417632
1     17.162807          14.721632        15.693973           27.411433
2     12.755851           6.851687        28.837764           20.000000
3     20.852981          14.353755        23.714740           23.778102
4     25.756818          13.885273        25.696677           24.117530


**Feature Engineering**


a) Socioeconomic Ratios

In [12]:
# Estimate average income based on poverty_rate and housing_quality_index
# Note: This estimation is arbitrary for illustration; use your own formula as needed
average_income_estimate = 50000 * (1 - (crime_data['poverty_rate'] / 100))
crime_data['income_to_police_budget_ratio'] = average_income_estimate / crime_data['police_budget']

# Calculate unemployment_population_ratio
crime_data['unemployment_population_ratio'] = crime_data['unemployment_rate'] / crime_data['population_density']

# Display new socioeconomic ratios to confirm
print("\nSocioeconomic Ratios:")
print(crime_data[['income_to_police_budget_ratio', 'unemployment_population_ratio']].head())



Socioeconomic Ratios:
   income_to_police_budget_ratio  unemployment_population_ratio
0                       0.027579                       0.000674
1                       0.030692                       0.010825
2                       0.034930                       0.001163
3                       0.015931                       0.001152
4                       0.021510                       0.001178


b) Spatial Density Features

In [13]:
# Assuming an estimated city area (e.g., 50 square miles for all cities here as an example)
city_area = 50

# Calculate crime rate per square mile
crime_data['crime_rate_per_sq_mile'] = crime_data['crime_rate'] / city_area

# Estimate number of police officers and calculate police officers per square mile
# Assuming an average cost per officer of $100,000
average_cost_per_officer = 100000
crime_data['police_officer_per_sq_mile'] = (crime_data['police_budget'] / average_cost_per_officer) / city_area

# Display spatial density features to confirm
print("\nSpatial Density Features:")
print(crime_data[['crime_rate_per_sq_mile', 'police_officer_per_sq_mile']].head())


Spatial Density Features:
   crime_rate_per_sq_mile  police_officer_per_sq_mile
0                1.205348                    0.263524
1                0.925109                    0.269895
2                0.685971                    0.249770
3                1.205348                    0.496806
4                1.204350                    0.345157


**Outlier Detection and Handling**

In [17]:
# Outlier Detection and Handling using IQR

# Define a function to cap outliers based on IQR
def cap_outliers(df, column):
    # Calculate Q1 (25th percentile) and Q3 (75th percentile)
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1  # Calculate IQR
    # Define the bounds for detecting outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    # Cap values outside the bounds
    df[column] = np.where(df[column] < lower_bound, lower_bound,
                          np.where(df[column] > upper_bound, upper_bound, df[column]))

# Columns to check for outliers
outlier_columns = ['housing_quality_index', 'drug_abuse_rate', 'crime_rate']

# Apply outlier capping to each specified column
for col in outlier_columns:
    cap_outliers(crime_data, col)

# Display the result after capping to verify
print("\nAfter handling outliers:")
print(crime_data[outlier_columns].describe())






After handling outliers:
       housing_quality_index  drug_abuse_rate  crime_rate
count             100.000000       100.000000  100.000000
mean                6.333993        24.905221   42.581484
std                 1.776006         5.212954   11.259347
min                 3.808572        15.693973   22.804727
25%                 5.207443        20.467628   35.215670
50%                 5.915373        25.552127   42.887745
75%                 7.246405        27.986140   51.755302
max                10.304847        34.389339   60.267405


**Assemble Features**

In [18]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import col

# Initialize Spark session
spark = SparkSession.builder.master("local[1]").appName("Feature Engineering").getOrCreate()

# Convert the pandas DataFrame to a PySpark DataFrame
crime_data_spark = spark.createDataFrame(crime_data)

# List of columns to include in the feature vector
feature_columns = [
    'population_density', 'poverty_rate', 'police_budget', 'unemployment_rate',
    'housing_quality_index', 'traffic_violation_rate', 'drug_abuse_rate',
    'mental_health_services', 'gun_ownership_rate', 'crime_rate',
    'income_to_police_budget_ratio', 'unemployment_population_ratio', 'crime_rate_per_sq_mile',
    'police_officer_per_sq_mile'
]

# Assemble the feature vector
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

# Apply the transformation
crime_data_spark = assembler.transform(crime_data_spark)

# Show a few rows of the transformed DataFrame
crime_data_spark.select('city_name', 'features').show(5, truncate=False)


+---------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|city_name|features                                                                                                                                                                                                                                     |
+---------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|City1    |[16295.0,27.321477096152144,1317619.0,10.986044887176178,5.7821753394707995,301.2138974596086,21.255301596104538,10.0,30.4176320872554,60.26740478168022,0.027579491075890625,6.741972928613794E-4,1.2053480956336045,0.2635238]             |


**Task 2: Train Baseline Model and Evaluate**

In [19]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import col

# 1. Data Splitting (80% Training, 20% Testing)
train_data, test_data = crime_data_spark.randomSplit([0.8, 0.2], seed=42)

# 2. Train a Linear Regression Model
lr = LinearRegression(featuresCol='features', labelCol='crime_rate')

# Fit the model on the training data
lr_model = lr.fit(train_data)

# 3. Make Predictions on the Test Set
test_predictions = lr_model.transform(test_data)

# 4. Compute Evaluation Metrics

# RMSE (Root Mean Squared Error)
evaluator_rmse = RegressionEvaluator(predictionCol="prediction", labelCol="crime_rate", metricName="rmse")
rmse = evaluator_rmse.evaluate(test_predictions)

# R-squared (R²)
evaluator_r2 = RegressionEvaluator(predictionCol="prediction", labelCol="crime_rate", metricName="r2")
r2 = evaluator_r2.evaluate(test_predictions)

# MAE (Mean Absolute Error)
evaluator_mae = RegressionEvaluator(predictionCol="prediction", labelCol="crime_rate", metricName="mae")
mae = evaluator_mae.evaluate(test_predictions)

# 5. Display Evaluation Metrics
print(f"Evaluation Metrics:")
print(f"RMSE: {rmse:.2f}")
print(f"R²: {r2:.2f}")
print(f"MAE: {mae:.2f}")



Evaluation Metrics:
RMSE: 0.00
R²: 1.00
MAE: 0.00


**Task 3: Feature Selection and Interpretability**

1. Train the Decision Tree Regressor and Calculate Feature Importance

In [23]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline

# Step 1: Handle categorical columns (e.g., city_name)
# Index categorical columns using StringIndexer (for example, 'city_name')
indexer = StringIndexer(inputCol='city_name', outputCol='city_name_indexed')

# Step 2: Assemble numeric features excluding the target 'crime_rate'
assembler = VectorAssembler(inputCols=[col for col in crime_data_spark.columns if col not in ['crime_rate', 'city_name']],
                            outputCol='features')

# Step 3: Train a Decision Tree Regressor
dt_regressor = DecisionTreeRegressor(featuresCol='features', labelCol='crime_rate')

# Step 4: Create a pipeline with the indexer, assembler, and regressor
pipeline = Pipeline(stages=[indexer, assembler, dt_regressor])

# Step 5: Train the model (fit the pipeline)
model = pipeline.fit(crime_data_spark)

# Step 6: Extract the trained Decision Tree model
dt_model = model.stages[-1]

# Step 7: Get feature importances
feature_importances = dt_model.featureImportances.toArray()

# Step 8: Get the feature names (exclude 'crime_rate' and 'city_name')
features_list = [col for col in crime_data_spark.columns if col not in ['crime_rate', 'city_name']]

# Step 9: Create a DataFrame with feature names and their corresponding importance scores
import pandas as pd

feature_importance_df = pd.DataFrame({
    'Feature': features_list,
    'Importance': feature_importances
})

# Step 10: Sort the DataFrame by importance to get the top features
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Step 11: Display the top 5 most important features
top_5_features = feature_importance_df.head(5)
print("Top 5 Features from Decision Tree Regressor:")
print(top_5_features)


IllegalArgumentException: Output column features already exists.